In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data-Copy1/penguins.csv')

In [3]:
df

,species,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181,3750,MALE
1,Adelie,Torgersen,39.5,17.4,186,3800,FEMALE
2,Adelie,Torgersen,40.3,18.0,195,3250,FEMALE
3,Adelie,Torgersen,36.7,19.3,193,3450,FEMALE
4,Adelie,Torgersen,39.3,20.6,190,3650,MALE
...,...,...,...,...,...,...,...
328,Gentoo,Biscoe,47.2,13.7,214,4925,FEMALE
329,Gentoo,Biscoe,46.8,14.3,215,4850,FEMALE
330,Gentoo,Biscoe,50.4,15.7,222,5750,MALE
331,Gentoo,Biscoe,45.2,14.8,212,5200,FEMALE


In [4]:
y = df['species']
x = df.iloc[:,1:]

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [6]:
le = LabelEncoder() 
le.fit(x['sex'])
le.classes_
new_sex = le.transform(x['sex'])
x['sex'] = new_sex

In [7]:
le = LabelEncoder()
le.fit(x['island'])
le.classes_
new_island = le.transform(x['island'])
x['island'] = new_island

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x, y,random_state = 80, test_size=0.2)

In [9]:
from sklearn.neural_network import MLPClassifier

In [10]:
clf = MLPClassifier(hidden_layer_sizes=(20, 20, 20), random_state=72)

In [11]:
clf.fit(x_train, y_train)

MLPClassifier(hidden_layer_sizes=(20, 20, 20), random_state=72)

In [12]:
y_predict = clf.predict(x_test)

In [13]:
from sklearn.metrics import accuracy_score

In [14]:
accuracy_score(a,y_predict)*100

70.1492537313433

In [15]:
df_male = df[df['sex'] == 'MALE'] # go delam data frame-ot na mashki i zhenski pingvini
df_female = df[df['sex'] == 'FEMALE']

In [16]:
y_male_df = df_male['species'] 
x_male_df = df_male.iloc[:,1:]
y_female_df = df_female['species']
x_female_df = df_female.iloc[:,1:]

In [17]:
x_male_df['island'] = le.fit_transform(x_male_df['island'])
x_male_df['sex'] = le.fit_transform(x_male_df['sex'])
x_female_df['sex'] = le.fit_transform(x_female_df['sex'])
x_female_df['island'] = le.fit_transform(x_female_df['island'])

In [18]:
y_predict_male = clf.predict(x_male_df)

In [19]:
accuracy_score(y_male_df,y_predict_male)*100

61.904761904761905

In [20]:
y_predict_female = clf.predict(x_female_df)

In [21]:
accuracy_score(y_female_df,y_predict_female)*100

79.39393939393939

In [22]:
data = {
    'island' : [2], 
    'culmen_length_mm': [72],
    'culmen_depth_mm': [34],
    'flipper_length_mm': [202],
    'body_mass_g' : [4450],
    'sex' : [1]
}

In [23]:
df_check = pd.DataFrame(data)

In [24]:
df_check

,island,culmen_length_mm,culmen_depth_mm,flipper_length_mm,body_mass_g,sex
0,2,72,34,202,4450,1


In [25]:
new_penguin = clf.predict(df_check)

In [26]:
new_penguin

array(['Adelie'], dtype='<U9')

In [27]:
import numpy as np

In [28]:
def entropy(a, b, c):
    """
    Function that calculates information entropy.
    :param a: subset A size
    :param b: subset B size
    :returns: entropy
    """
    if a == 0 or b == 0 or c == 0:
        return 0
    m = a + b
    return - a/m * np.log2(a/m) - b/m * np.log2(b/m) -c/m * np.log2(c/m)

In [29]:
# prvo ja prajme entropy funkcijia
def entropy_of_dataset(df, output_column, class_1, class_2, class_3):
#     ги земаме податоците од множеството df каде излезот е еднаков на класа 1
    class_1_data = df.query(f"{output_column} == '{class_1}'")
#     ги земаме податоците од множеството df каде излезот е еднаков на класа 2
    class_2_data = df.query(f"{output_column} == '{class_2}'")

    class_3_data = df.query(f"{output_column} == '{class_3}'")
#     ја пресметуваме ентропијата за даденото множество податоци
    return entropy(class_1_data.shape[0], class_2_data.shape[0], class_3_data.shape[0])

In [30]:
output_column = 'species'
class_1 = 'Adelie'
class_2 = 'Gentoo'
class_3 = 'Chinstrap'

h_before_split = entropy_of_dataset(df,output_column,class_1, class_2, class_3)

In [31]:
df['culmen_length_mm_over_30'] = df['culmen_length_mm'] > 40
df = df [['culmen_length_mm_over_30','sex','species']]

In [32]:
# rabotime prvo za SEX
male_data = df.query("sex == 'MALE'")
female_data = df.query("sex == 'FEMALE'")
h_male_data = entropy_of_dataset(male_data,output_column,class_1, class_2, class_3)
h_female_data = entropy_of_dataset(female_data,output_column,class_1, class_2, class_3)
factor_male_data = male_data.shape[0]/df.shape[0] # gi zema prvite i gi deli so prvite nekolku
factor_female_data = female_data.shape[0]/df.shape[0]

h_after_split_sex = factor_male_data*h_male_data + factor_female_data*h_female_data

information_gain_sex = h_before_split - h_after_split_sex
information_gain_sex

0.00013793765131220326

In [33]:
# rabotime za CULMENLENGHT
true_data = df.query("culmen_length_mm_over_30 == True") #true e culmen over 30
false_data = df.query("culmen_length_mm_over_30 == False") #false culmen under 30
h_true_data = entropy_of_dataset(true_data,output_column,class_1, class_2, class_3)
h_false_data = entropy_of_dataset(false_data,output_column,class_1, class_2, class_3)
factor_true_data = true_data.shape[0]/df.shape[0] # gi zema prvite i gi deli so prvite nekolku
factor_false_data = false_data.shape[0]/df.shape[0]

h_after_split_culmen = factor_true_data*h_true_data + factor_false_data*h_false_data

information_gain_culmen = h_before_split - h_after_split_culmen
information_gain_culmen

0.49635275845708526